In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}")
spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {schema}")

DataFrame[]

In [0]:
# Get all tables under the specified catalog and schema
tables_df = spark.sql(f"SHOW TABLES IN {catalog}.{schema}")
table_names = [row['tableName'] for row in tables_df.collect()]

# Delete all records from each table
for table in table_names:
    spark.sql(f"DROP TABLE IF EXISTS {catalog}.{schema}.{table}")

# Display the list of tables that were cleared
display(spark.createDataFrame([(f"{catalog}.{schema}.{t}",) for t in table_names], ["Dropped_Table"]))

Dropped_Table
shared.yash_underwriter_demo.actuarial
shared.yash_underwriter_demo.client_submission
shared.yash_underwriter_demo.quote
shared.yash_underwriter_demo.quote_status
shared.yash_underwriter_demo.risk


In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog}.{schema}.actuarial (
  Product_Type STRING COMMENT 'Type of life insurance product (e.g., Term Life, Whole Life, Universal Life)',
  Base_Mortality_Rate DECIMAL(10, 6) COMMENT 'Base mortality rate per 1000 lives, derived from actuarial tables',
  Age_Factor DOUBLE COMMENT 'Adjustment factor based on applicant age group',
  Gender_Factor DOUBLE COMMENT 'Adjustment factor based on gender',
  Smoker_Factor DOUBLE COMMENT 'Adjustment factor for smoker vs non-smoker status',
  Health_Class STRING COMMENT 'Health classification (Preferred, Standard, Substandard)',
  Reinsurance_Cost DECIMAL(10, 6) COMMENT 'Portion of risk cost ceded to reinsurer'
)
USING delta
COMMENT 'Contains actuarial rate factors used to calculate base premiums for life insurance products, incorporating age, gender, and health-based adjustments.'
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog}.{schema}.client_submission (
  Client_ID STRING,
  First_Name STRING,
  Last_Name STRING,
  Date_of_Birth DATE,
  Gender STRING,
  Occupation STRING,
  Annual_Income_USD BIGINT,
  Smoking_Status STRING,
  Medical_Conditions STRING,
  Family_History STRING,
  Coverage_Amount_Requested BIGINT,
  Policy_Type STRING,
  Location STRING,
  Risk_Level STRING,
  Submission_Date DATE
)
USING delta
COMMENT 'Captures all initial application data from individual life insurance applicants, including demographics, health, and lifestyle details used for underwriting assessment.'
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog}.{schema}.quote (
  Quote_ID STRING,
  Client_ID STRING,
  Coverage_Amount BIGINT,
  Premium_Estimate DOUBLE,
  Term_Length_Years INT,
  Product_Option STRING,
  Health_Rating STRING,
  Terms_and_Conditions_Summary STRING
)
USING delta
COMMENT 'Stores premium quotes generated for applicants based on actuarial factors and underwriting rules. Each quote corresponds to a unique client submission.'
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog}.{schema}.quote_status (
  Quote_ID STRING,
  Client_ID STRING,
  Stage STRING COMMENT 'Application stage (e.g., Under Review, Medical Exam Required, Approved, Issued, Declined)',
  Status STRING COMMENT 'Current status of the quote',
  Underwriter_Assigned STRING COMMENT 'Name or ID of the underwriter handling the case',
  Comments STRING
)
USING delta
COMMENT 'Tracks the current processing and decision status for each life insurance quote, including underwriter assignment and review notes.'
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog}.{schema}.risk (
  Client_ID STRING COMMENT 'Unique identifier for the applicant.',
  Age INT COMMENT 'Age of the applicant at time of submission.',
  BMI DOUBLE COMMENT 'Body Mass Index value used in health risk evaluation.',
  Blood_Pressure STRING COMMENT 'Blood pressure reading category (e.g., Normal, Elevated, Stage 1 Hypertension).',
  Cholesterol_Level STRING COMMENT 'Cholesterol level classification (e.g., Normal, Borderline, High).',
  Smoking_Status STRING COMMENT 'Smoker or Non-Smoker, affects mortality risk.',
  Alcohol_Use STRING COMMENT 'Reported alcohol consumption (None, Moderate, Heavy).',
  Exercise_Frequency STRING COMMENT 'Frequency of exercise (e.g., Daily, Weekly, Rarely).',
  Medical_History_Score DOUBLE COMMENT 'Aggregated score from known medical conditions.',
  Family_History_Risk DOUBLE COMMENT 'Risk factor from hereditary conditions.',
  Lifestyle_Risk DOUBLE COMMENT 'Composite lifestyle-based risk score.',
  Overall_Risk_Score DOUBLE COMMENT 'Calculated overall underwriting risk score combining health, lifestyle, and hereditary risk factors.'
)
USING delta
COMMENT 'Contains risk assessment metrics for individual life insurance applicants. Combines demographic, health, and lifestyle data to produce an overall underwriting risk score used in pricing and decision models.'
""")

DataFrame[]

In [0]:
spark.sql(f"""
INSERT INTO {catalog}.{schema}.actuarial VALUES
('Term Life', 0.0012, 1.0, 1.05, 1.3, 'Preferred', 0.0004),
('Term Life', 0.0015, 1.2, 1.0, 1.5, 'Standard', 0.0006),
('Whole Life', 0.0021, 1.1, 1.1, 1.4, 'Preferred', 0.0007),
('Whole Life', 0.0028, 1.3, 1.0, 1.6, 'Standard', 0.0010),
('Universal Life', 0.0023, 1.1, 1.1, 1.4, 'Preferred', 0.0008),
('Universal Life', 0.0029, 1.3, 1.0, 1.6, 'Standard', 0.0011)
""")

spark.sql(f"""
INSERT INTO {catalog}.{schema}.client_submission VALUES
('C001','Alice','Nguyen','1988-03-22','Female','Accountant',85000,'Non-Smoker','None','None',500000,'Term Life','Seattle','Low','2025-01-15'),
('C002','Brian','Lopez','1975-06-12','Male','Truck Driver',68000,'Smoker','Hypertension','Cardiac',250000,'Whole Life','Houston','High','2025-02-10'),
('C003','Chloe','Patel','1993-09-08','Female','Software Engineer',120000,'Non-Smoker','None','None',1000000,'Term Life','Austin','Low','2025-03-05'),
('C004','Daniel','Kim','1980-02-14','Male','Sales Manager',95000,'Non-Smoker','Diabetes','Diabetes',750000,'Universal Life','Denver','Medium','2025-03-22'),
('C005','Ella','Reed','1995-11-30','Female','Nurse',72000,'Non-Smoker','Asthma','None',400000,'Term Life','Miami','Low','2025-04-12'),
('C006','Frank','Wright','1965-04-11','Male','Electrician',78000,'Smoker','Heart Disease','Cardiac',300000,'Whole Life','Chicago','High','2025-05-08'),
('C007','Grace','Olsen','1989-07-18','Female','Marketing Specialist',98000,'Non-Smoker','None','None',600000,'Universal Life','Boston','Low','2025-05-27'),
('C008','Henry','Diaz','1972-05-22','Male','Chef',64000,'Smoker','Obesity','Diabetes',200000,'Whole Life','Phoenix','High','2025-06-14'),
('C009','Isabella','Lee','1986-01-09','Female','Teacher',71000,'Non-Smoker','Hypertension','Hypertension',500000,'Term Life','San Diego','Medium','2025-06-28'),
('C010','Jack','Taylor','1990-10-19','Male','Software Developer',130000,'Non-Smoker','None','None',900000,'Universal Life','San Jose','Low','2025-07-05'),
('C011','Karen','Martinez','1982-08-02','Female','Financial Advisor',115000,'Non-Smoker','None','Cardiac',750000,'Whole Life','Dallas','Medium','2025-07-18'),
('C012','Leo','Singh','1970-12-17','Male','Mechanic',60000,'Smoker','Hypertension','Diabetes',250000,'Whole Life','Detroit','High','2025-08-02'),
('C013','Mia','Adams','1996-05-03','Female','Graphic Designer',85000,'Non-Smoker','None','None',450000,'Term Life','Portland','Low','2025-08-19'),
('C014','Noah','Brown','1983-09-25','Male','Engineer',100000,'Non-Smoker','Diabetes','None',800000,'Universal Life','Atlanta','Medium','2025-09-01'),
('C015','Olivia','Johnson','1978-07-12','Female','HR Manager',92000,'Non-Smoker','Hypertension','Cardiac',600000,'Whole Life','Nashville','Medium','2025-09-20'),
('C016','Paul','Gomez','1992-11-01','Male','Pilot',140000,'Non-Smoker','None','None',1500000,'Term Life','New York','Low','2025-09-30'),
('C017','Quinn','Evans','1969-03-27','Male','Construction Foreman',75000,'Smoker','Heart Disease','Cardiac',300000,'Whole Life','Cleveland','High','2025-10-05'),
('C018','Riley','Chen','1987-12-09','Female','Pharmacist',110000,'Non-Smoker','None','Diabetes',850000,'Universal Life','San Francisco','Medium','2025-10-10'),
('C019','Sam','Parker','1981-10-05','Male','Lawyer',160000,'Non-Smoker','None','None',1200000,'Whole Life','Washington','Low','2025-10-25'),
('C020','Tina','Hall','1976-02-15','Female','Teacher',67000,'Non-Smoker','Asthma','Hypertension',400000,'Term Life','Tampa','Medium','2025-11-09'),
('C021','Uma','Rao','1984-06-21','Female','Entrepreneur',200000,'Non-Smoker','None','None',1800000,'Universal Life','Los Angeles','Low','2025-11-23'),
('C022','Victor','Allen','1973-04-04','Male','Warehouse Worker',55000,'Smoker','Obesity','Cardiac',1250000,'Whole Life','Cincinnati','High','2025-12-05'),
('C023','Wendy','Scott','1991-09-11','Female','UX Designer',95000,'Non-Smoker','None','None',650000,'Term Life','Charlotte','Low','2025-12-20')
""")

spark.sql(f"""
INSERT INTO {catalog}.{schema}.risk VALUES
('C001',37,22.3,'Normal','Normal','Non-Smoker','None','Daily',1.2,0.8,2.5,2.1),
('C002',50,29.4,'High','High','Smoker','Moderate','Rarely',3.5,3.0,8.0,6.5),
('C003',32,23.1,'Normal','Normal','Non-Smoker','None','Daily',0.9,1.0,2.2,2.0),
('C004',45,27.8,'Elevated','Borderline','Non-Smoker','Moderate','Weekly',2.5,2.2,5.0,4.1),
('C005',30,21.5,'Normal','Normal','Non-Smoker','None','Daily',1.0,0.7,2.0,1.8),
('C006',60,31.6,'High','High','Smoker','Heavy','Rarely',4.5,3.8,9.5,7.8),
('C007',36,24.2,'Normal','Normal','Non-Smoker','Moderate','Daily',1.1,0.8,2.3,2.0),
('C008',53,33.5,'High','Borderline','Smoker','Heavy','Rarely',3.8,3.1,8.5,6.9),
('C009',39,25.9,'Elevated','Borderline','Non-Smoker','Moderate','Weekly',2.2,2.0,4.5,3.8),
('C010',35,23.5,'Normal','Normal','Non-Smoker','None','Daily',1.0,1.1,2.0,2.0),
('C011',43,26.1,'Elevated','Borderline','Non-Smoker','Moderate','Weekly',2.0,2.2,4.2,3.7),
('C012',55,30.8,'High','High','Smoker','Heavy','Rarely',4.2,3.5,9.2,7.6),
('C013',29,22.0,'Normal','Normal','Non-Smoker','None','Daily',1.1,0.9,2.3,2.0),
('C014',42,27.0,'Elevated','Borderline','Non-Smoker','Moderate','Weekly',2.4,2.1,4.6,3.9),
('C015',47,26.7,'Elevated','High','Non-Smoker','Moderate','Weekly',2.8,2.5,5.8,4.8),
('C016',33,24.5,'Normal','Normal','Non-Smoker','None','Daily',1.0,0.9,2.0,1.9),
('C017',56,31.8,'High','High','Smoker','Heavy','Rarely',4.8,3.9,9.8,8.0),
('C018',38,25.0,'Normal','Borderline','Non-Smoker','Moderate','Weekly',2.0,1.8,4.0,3.5),
('C019',44,26.4,'Normal','Normal','Non-Smoker','Moderate','Daily',1.5,1.5,3.5,3.0),
('C020',49,27.5,'Elevated','Borderline','Non-Smoker','Moderate','Weekly',2.5,2.0,5.0,4.1),
('C021',41,25.3,'Normal','Normal','Non-Smoker','None','Daily',1.1,1.0,2.3,2.0),
('C022',52,33.0,'High','High','Smoker','Heavy','Rarely',4.0,3.5,8.8,7.3),
('C023',34,23.2,'Normal','Normal','Non-Smoker','None','Daily',1.0,1.0,2.1,1.9)
""")

spark.sql(f"""
INSERT INTO {catalog}.{schema}.quote VALUES
('Q001','C001',500000,610.25,20,'Term Life','Low','Preferred terms. No medical exam required.'),
('Q002','C002',250000,2050.40,15,'Whole Life','High','High-risk applicant. Subject to medical exam.'),
('Q003','C003',1000000,920.10,20,'Term Life','Low','Preferred terms. No medical exam required.'),
('Q004','C004',750000,1325.50,20,'Universal Life','Medium','Standard underwriting terms.'),
('Q005','C005',400000,550.60,15,'Term Life','Low','Preferred terms. No medical exam required.'),
('Q006','C006',300000,2300.75,20,'Whole Life','High','High-risk applicant. Subject to medical exam.'),
('Q007','C007',600000,790.80,20,'Universal Life','Low','Preferred terms. No medical exam required.'),
('Q008','C008',200000,1850.20,15,'Whole Life','High','High-risk applicant. Subject to medical exam.'),
('Q009','C009',500000,950.45,20,'Term Life','Medium','Standard underwriting terms.'),
('Q010','C010',900000,910.90,20,'Universal Life','Low','Preferred terms.'),
('Q011','C011',750000,1100.70,20,'Whole Life','Medium','Standard underwriting terms.'),
('Q012','C012',250000,2150.85,15,'Whole Life','High','High-risk applicant.'),
('Q013','C013',450000,590.35,15,'Term Life','Low','Preferred terms.'),
('Q014','C014',800000,1250.60,20,'Universal Life','Medium','Standard underwriting terms.'),
('Q015','C015',600000,1150.80,20,'Whole Life','Medium','Standard underwriting terms.'),
('Q016','C016',1500000,980.20,20,'Term Life','Low','Preferred terms.'),
('Q017','C017',300000,2400.40,15,'Whole Life','High','High-risk applicant.'),
('Q018','C018',850000,1180.10,20,'Universal Life','Medium','Standard underwriting terms.'),
('Q019','C019',1200000,870.25,20,'Whole Life','Low','Preferred terms.'),
('Q020','C020',400000,1020.65,15,'Term Life','Medium','Standard underwriting terms.'),
('Q021','C021',1800000,990.80,20,'Universal Life','Low','Preferred terms.'),
('Q022','C022',1250000,2100.75,15,'Whole Life','High','High-risk applicant.'),
('Q023','C023',650000,670.50,20,'Term Life','Low','Preferred terms.')
""")

spark.sql(f"""
INSERT INTO {catalog}.{schema}.quote_status VALUES
('Q001','C001','Approved','Completed','Jessica Moore','Approved after standard underwriting.'),
('Q002','C002','Under Review','Pending','David Lee','Medical exam requested.'),
('Q003','C003','Issued','Completed','Sarah Patel','Preferred approval.'),
('Q004','C004','Under Review','In Progress','Michael Chen','Additional labs required.'),
('Q005','C005','Approved','Completed','Emma Jones','Clean record.'),
('Q006','C006','Declined','Closed','Robert Hill','High cardiac risk.'),
('Q007','C007','Issued','Completed','John Smith','Approved preferred class.'),
('Q008','C008','Under Review','Pending','Angela Davis','BMI and BP concerns.'),
('Q009','C009','Medical Exam Required','In Progress','Kevin Reed','BP borderline.'),
('Q010','C010','Approved','Completed','Rachel Adams','Issued without rider.'),
('Q011','C011','Under Review','In Progress','Chris White','Waiting on labs.'),
('Q012','C012','Declined','Closed','Amy Turner','Severe hypertension.'),
('Q013','C013','Issued','Completed','Hannah Kim','Preferred approval.'),
('Q014','C014','Medical Exam Required','In Progress','Eric Johnson','A1C pending.'),
('Q015','C015','Under Review','Pending','Maria Lopez','BP follow-up required.'),
('Q016','C016','Issued','Completed','James Brown','Clean aviation profile.'),
('Q017','C017','Declined','Closed','Tom Wilson','Multiple cardiac issues.'),
('Q018','C018','Approved','Completed','Sophia Green','Standard approval.'),
('Q019','C019','Issued','Completed','Olivia Martinez','Premium confirmed.'),
('Q020','C020','Medical Exam Required','In Progress','Daniel Evans','Asthma check pending.'),
('Q021','C021','Issued','Completed','Patrick Young','Preferred elite class.'),
('Q022','C022','Declined','Closed','Laura Scott','Obesity and smoker profile.'),
('Q023','C023','Approved','Completed','Ben Carter','Preferred non-smoker.')
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION {catalog}.{schema}.get_quote(client STRING)
RETURNS TABLE (
  Quote_ID STRING,
  Client STRING,
  Coverage_Amount BIGINT,
  Premium_Estimate DOUBLE,
  Term_Length_Years INT,
  Product_Option STRING,
  Health_Rating STRING,
  Terms_and_Conditions_Summary STRING
)
COMMENT 'Returns all quote details for a specified client (input is full name: First Last), including coverage, premium estimate, product option, and health rating.'
RETURN
  SELECT 
    Quote_ID, 
    CONCAT(c.First_Name, ' ', c.Last_Name) AS Client,
    Coverage_Amount,
    Premium_Estimate,
    Term_Length_Years,
    Product_Option,
    Health_Rating,
    Terms_and_Conditions_Summary
  FROM {catalog}.{schema}.quote q
  JOIN {catalog}.{schema}.client_submission c
    ON q.Client_ID = c.Client_ID
  WHERE LOWER(CONCAT(c.First_Name, ' ', c.Last_Name)) = LOWER(client);
""")

DataFrame[]

In [0]:
# Step 1: Get a sample client full name
client_name_row = spark.sql(f"""
    SELECT CONCAT(First_Name, ' ', Last_Name) AS Client_Name
    FROM {catalog}.{schema}.client_submission
    LIMIT 1
""").collect()[0]
client_name = client_name_row['Client_Name']

# Step 2: Use the value to test get_quote_status function
display(spark.sql(f"""
    SELECT q.* FROM {catalog}.{schema}.get_quote('{client_name}') q
"""))

Quote_ID,Client,Coverage_Amount,Premium_Estimate,Term_Length_Years,Product_Option,Health_Rating,Terms_and_Conditions_Summary
Q001,Alice Nguyen,500000,610.25,20,Term Life,Low,Preferred terms. No medical exam required.


In [0]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION {catalog}.{schema}.get_quote_status(client STRING)
RETURNS TABLE (
  Quote_ID STRING,
  Client STRING,
  Stage STRING,
  Status STRING,
  Underwriter_Assigned STRING,
  Comments STRING
)
COMMENT 'Returns the current quote status, application stage, underwriter assignment, and comments for a specified client (input is full name: First Last).'
RETURN
  SELECT 
    q.Quote_ID,
    CONCAT(c.First_Name, ' ', c.Last_Name) AS Client,
    q.Stage,
    q.Status,
    q.Underwriter_Assigned,
    q.Comments
  FROM {catalog}.{schema}.quote_status q
  JOIN {catalog}.{schema}.client_submission c
    ON q.Client_ID = c.Client_ID
  WHERE LOWER(CONCAT(c.First_Name, ' ', c.Last_Name)) = LOWER(client);
""")

DataFrame[]

In [0]:
# Step 1: Get a sample client full name
client_name_row = spark.sql(f"""
    SELECT CONCAT(First_Name, ' ', Last_Name) AS Client_Name
    FROM {catalog}.{schema}.client_submission
    LIMIT 1
""").collect()[0]
client_name = client_name_row['Client_Name']

# Step 2: Use the value to test get_quote_status function
display(spark.sql(f"""
    SELECT q.* FROM {catalog}.{schema}.get_quote_status('{client_name}') q
"""))

Quote_ID,Client,Stage,Status,Underwriter_Assigned,Comments
Q001,Alice Nguyen,Approved,Completed,Jessica Moore,Approved after standard underwriting.


In [0]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION {catalog}.{schema}.get_risk_score(client STRING)
RETURNS TABLE (
  Client STRING,
  Age INT,
  BMI DOUBLE,
  Blood_Pressure STRING,
  Cholesterol_Level STRING,
  Smoking_Status STRING,
  Alcohol_Use STRING,
  Exercise_Frequency STRING,
  Medical_History_Score DOUBLE,
  Family_History_Risk DOUBLE,
  Lifestyle_Risk DOUBLE,
  Overall_Risk_Score DOUBLE
)
COMMENT 'Returns detailed risk assessment metrics for a specified client, including age, BMI, blood pressure, cholesterol, lifestyle, and overall underwriting risk score. Input is the full client name (First Last).'
RETURN
  SELECT 
    CONCAT(c.First_Name, ' ', c.Last_Name) AS Client,
    q.Age,
    q.BMI,
    q.Blood_Pressure,
    q.Cholesterol_Level,
    q.Smoking_Status,
    q.Alcohol_Use,
    q.Exercise_Frequency,
    q.Medical_History_Score,
    q.Family_History_Risk,
    q.Lifestyle_Risk,
    q.Overall_Risk_Score
  FROM {catalog}.{schema}.risk q
  JOIN {catalog}.{schema}.client_submission c
    ON q.Client_ID = c.Client_ID
  WHERE LOWER(CONCAT(c.First_Name, ' ', c.Last_Name)) = LOWER(client);
""")

DataFrame[]

In [0]:
# Step 1: Get a sample client full name
client_name_row = spark.sql(f"""
    SELECT CONCAT(First_Name, ' ', Last_Name) AS Client_Name
    FROM {catalog}.{schema}.client_submission
    LIMIT 1
""").collect()[0]
client_name = client_name_row['Client_Name']

# Step 2: Use the value to test get_quote_status function
display(spark.sql(f"""
    SELECT q.* FROM {catalog}.{schema}.get_risk_score('{client_name}') q
"""))

Client,Age,BMI,Blood_Pressure,Cholesterol_Level,Smoking_Status,Alcohol_Use,Exercise_Frequency,Medical_History_Score,Family_History_Risk,Lifestyle_Risk,Overall_Risk_Score
Alice Nguyen,37,22.3,Normal,Normal,Non-Smoker,None,Daily,1.2,0.8,2.5,2.1


### 🚀 How to Use in Databricks Genie

Here are example Genie prompts for your underwriting demo:

**Prompt	Genie’s Possible Response**

“Explain why this client’s premium is $950.”	“80% of the increase is due to smoking status (+50%), 15% from above-average risk score, and 5% from age.”

“Show top 10 applicants where smoking had the largest impact.”	Genie lists those with highest Premium_from_Smoking.

“Compare average explained premiums between genders.”	Genie produces an aggregated view.

“Which factor contributes most to premium variance across all applicants?”	Genie visualizes variance drivers across Age, Smoking, Risk.

**Then you can ask Genie things like**:

“Compare average premium factors by pricing tier.”
“Which group has the highest smoking-related premium impact?”

